## 코드


# 1. 


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms #필요한 라이브러리 임포트 시키기
 
device = 'cuda' if torch.cuda.is_available() else 'cpu' #gpu 
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")
 
learning_rate = 0.001 #러닝레이트
batch_size = 100 #배치사이즈 100
num_classes = 10 #클래스 개수가 10개
epochs = 5 #5에폭입니다.

# 2.

In [ ]:
train_set = torchvision.datasets.MNIST( #엠니스트 데이터셋 로딩하기
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() #텐셔형태로 변환
    ])
)
test_set = torchvision.datasets.MNIST( #테스트 데이터셋 로딩하기
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()#텐셔형태로 변환
    ])
)

# 3.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size) # 배치사이즈 100에 맞게 데이터 로드하기
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size) #테스트 셋도 위와 마찬가지로
 
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples) #배치, 데이터, 타겟값을 다음 데이터로 뽑아주기
example_data.shape #형태 확인

# 4.

In [ ]:
class ConvNet(nn.Module): #convnet이라는 클래스 만들기
  def __init__(self): 
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #  인풋채널사이즈= 1, 아웃풋 채널 사이즈= 10, kernel size = 5, stride = 1, 패딩=0 컨볼루션 연산 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) #  인풋채널사이즈= 10, 아웃풋 채널 사이즈= 20, kernel size = 5, stride = 1, 패딩=0 컨볼루션
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) #드롭아웃 비율 0.25(없는애 취급하고 학습)
        self.mp = nn.MaxPool2d(2)  #가장 큰값 2x2 에서 뽑기
        self.fc1 = nn.Linear(320,100)  #인풋사이즈 320->아웃풋 100: linear(선형회귀) 함수
        self.fc2 = nn.Linear(100,10)  #인풋사이즈 100->아웃풋 10: linear 함수

  def forward(self, x): #포워드 연산
        # 100,1,28,28 size
        x = F.relu(self.mp(self.conv1(x))) #relu는 활성화 비선형 함수
        # 100,10,12,12 size
        x = F.relu(self.mp(self.conv2(x))) 
        # 100,20,4,4 size
        x = self.drop2D(x) #위에 init에서 정의한 drop2d
        # 100,20,4,4 size
        x = x.view(x.size(0), -1) #x.size(0)shape로 자동지정-> 100,320 size
        x = self.fc1(x) #위에서 정의한 fc1 ->100,100 size
        x = self.fc2(x) #위에서 정의한 fc2 ->100,10 size
        return F.log_softmax(x) #소프트맥스에 로그값 취한 것

# 5.

In [ ]:
model = ConvNet().to(device)  #conv넷 모델로 지정
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) #옵티마이저로 아담 사용
 
for epoch in range(epochs):  #5에폭만큼 돌겠다.
    avg_cost = 0

    for data, target in train_loader: 
        data = data.to(device) #데이터 로드
        target = target.to(device) #타겟값 로드
        optimizer.zero_grad() 
        #  모든 model의 gradient 값을 0으로 설정 - 초기화 이유 : 
        #파이토치에서는 그래디언트값들을 추후에 backward를 해줄 때 계속 더해준다. 
        #그래서 백프로파게이션을 진행하기 전에 그래디언트를 zero로 만들어줘야한다.
        #안그러면 이상향 방향으로 학습된다
        hypothesis = model(data) #예측값
        cost = criterion(hypothesis, target) #크로스엔트로피 값. 손실계산
        cost.backward() #백프로파게이션-> w와 b가 계산됨
        optimizer.step()  # 옵티마이저의 step함수를 호출하여 인수로 들어갔던 w와 b에서 리턴되는
        #변수들의 기울기에 러닝레이트를 곱하여 빼준다.
        avg_cost += cost / len(train_loader) #코스트 합 계산
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
 

# 6.

In [ ]:
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)#데이터로드
        target = target.to(device) #타겟값 로드
        out = model(data) #모델의 아웃풋값
        preds = torch.max(out.data, 1)[1]  #타겟값
        total += len(target) #전체 데이터개수
        correct += (preds==target).sum().item()  #맞는 값
        
    print('Test Accuracy: ', 100.*correct/total, '%')

# 7. 

